In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [3]:
df_original = pd.read_csv('../data/_data.csv')
df = df_original.copy()
#  удаляем столбцы 'Unnamed: 0', 'Тип', 'Телефоны', 'Описание', 'Название ЖК', 'Серия дома', 'Мусоропровод','Ссылка на объявление', 'Лифт'
df.drop(columns=['Unnamed: 0', 'Тип', 'Телефоны', 'Описание', 'Название ЖК', 'Серия дома', 'Мусоропровод','Ссылка на объявление', 'Лифт'], inplace=True)

In [4]:
# в цене отбрасываем что после цены
def extract_first_number(val):
    if pd.isna(val):
        return np.nan
    val = str(val)
    # первое число (целое или с запятой или точкой)
    match = re.search(r'\d+[\d,.]*', val)
    if match:
        # оставляем только найденное число, удаляем пробелы, заменяем запятую на точку
        number_str = match.group(0).replace(' ', '').replace(',', '.')
        try:
            return float(number_str)
        except ValueError:
            return np.nan
    return np.nan

df['Цена'] = df['Цена'].apply(extract_first_number)

# в площади оставляем первую площадь (отбрасываем все до /)
def extract_total_area(val):
    if pd.isna(val):
        return None
    val = str(val) 
    return float(val.split('/')[0])

df['Площадь, м2'] = df['Площадь, м2'].apply(extract_total_area)

In [5]:
# удаляем время до метро
def extract_station(metro_str):
    if pd.isna(metro_str):
        return None
    # ищем "м. " и захватываем все до первой скобки или конца строки
    match = re.search(r'м\.\s*([^(\n\r]+)', metro_str)
    if match:
        # возвращаем название станции с обрезанием пробелов
        return match.group(1).strip()
    return metro_str  # Если не совпало, возвращаем исходное значение

df['Метро'] = df['Метро'].apply(extract_station)


In [6]:
# в столбце количество комнат оставляем только число комнат
def extract_room_number(value):
    if pd.isna(value):
        return None
    # Ищем первое число в строке
    match = re.search(r'\d+', str(value))
    if match:
        return int(match.group(0))
    return None

df['Количество комнат'] = df['Количество комнат'].apply(extract_room_number)


In [7]:
# заполняем пропуски
# Количество комнат
# Количество комнат преобразуем в число, заполним по медиане внутри групп по площади
df['Количество комнат'] = pd.to_numeric(df['Количество комнат'], errors='coerce')
medians = df.groupby('Площадь, м2')['Количество комнат'].transform('median')
df['Количество комнат'] = df['Количество комнат'].fillna(medians)
global_med = df['Количество комнат'].median()
df['Количество комнат'] = df['Количество комнат'].fillna(global_med).round().astype('Int64')
# метро отсутствующие меняем на "unknown"
df['Метро'] = df['Метро'].fillna('unknown')
# парковка отсутствующие меняем на "unknown"
df['Парковка'] = df['Парковка'].fillna('unknown')
# ремонт
df['Ремонт'] = df['Ремонт'].fillna(df['Ремонт'].mode()[0])
# площадь комнат
def extract_mean_room_area(val):
    if pd.isna(val):
        return None
    numbers = re.findall(r'\d+', str(val))
    if numbers:
        return np.mean([int(n) for n in numbers])
    return None
df['Площадь комнат, м2'] = df['Площадь комнат, м2'].apply(extract_mean_room_area)
df['Площадь комнат, м2'] = df['Площадь комнат, м2'].fillna(df['Площадь комнат, м2'].median())
# балкон
df['Балкон'] = df['Балкон'].fillna('unknown')
# окна
df['Окна'] = df['Окна'].fillna(df['Окна'].mode()[0])
# санузел
df['Санузел'] = df['Санузел'].fillna('unknown')
# Заполняем пустые значения в столбце "Можно с детьми/животными" 'unknown'
df['Можно с детьми/животными'] = df['Можно с детьми/животными'].fillna('unknown')
# Заполняем пустые значения в столбце "Дополнительно" 'unknown'
df['Дополнительно'] = df['Дополнительно'].fillna('unknown')
# Заполняем пустые значения в столбце "Высотка потолков" средним значением по датафрейму с удаленными выбросами(3.00)
df['Высота потолков, м'] = df['Высота потолков, м'].fillna(3.00)

In [8]:
# удаляем выбросы по цене
df_mean = df['Цена'].mean()
df_std = df['Цена'].std()
df = df[(df['Цена'] > df_mean - 0.5 * df_std) & (df['Цена'] < df_mean + 2 * df_std)]

In [9]:
# переименовываем столбцы
df.rename(columns={
    'ID  объявления': 'id',
    'Количество комнат': 'rooms',
    'Метро': 'metro',
    'Адрес': 'address',
    'Площадь, м2': 'area',
    'Дом': 'building_info',
    'Парковка': 'parking',
    'Цена': 'price',
    'Ремонт': 'renovation',
    'Площадь комнат, м2': 'room_area',
    'Балкон': 'balcony',
    'Окна': 'windows',
    'Санузел': 'bathroom',
    'Можно с детьми/животными': 'allowed_children_pets',
    'Дополнительно': 'additional',
    'Высота потолков, м': 'ceiling_height'
}, inplace=True)

In [ ]:
df.head(2)

,id,rooms,metro,address,area,building_info,parking,price,renovation,room_area,balcony,windows,bathroom,allowed_children_pets,additional,ceiling_height
4,273614615,2,Арбатская,"Москва, улица Новый Арбат, 15",58.0,"12/26, Панельный",unknown,225000.0,Евроремонт,19.0,unknown,На улицу и двор,Совмещенный (2),unknown,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.9
7,274475342,3,Смоленская,"Москва, улица Арбат, 43С3",98.0,"2/4, Монолитный",подземная,250000.0,Евроремонт,21.0,unknown,Во двор,"Совмещенный (1), Раздельный (1)","Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...",3.2


In [11]:
df.to_csv('data.csv', encoding='utf-8')